In [2]:
import pandas as pd
import numpy as np
import glob
import os
import subprocess
import matplotlib.pyplot as plt

In [3]:
# Things to do before running the pipeline:
# 1. After installing bismark AND trim_galore, create a symbolic link to it in your /usr/bin file
#   Ex. sudo ln -s /path/to/bismark_directory/bismark /usr/bin/bismark
#   Here is a list of commands that you'll have to do this for:
#   bismark
#   bismark_genome_preparation

In [4]:
target_dir = "/home/jwright/RRBS/RQ025464-Cortese"

In [5]:
import shutil

def check_executability(command):
    executable = shutil.which(command)
    if executable:
        return None
    else:
        raise PermissionError(f'The command, {command}, is not executable. Make sure that there is a symbolic link to /usr/bin') 


In [20]:
# Assign directories

subfolders = sorted(glob.glob(os.path.join(target_dir,'RS-*')))

trim_galore_dir_list = []
nugen_trim_dir_list = []
bismark_dir_list = []
dedup_dir_list = []

for subfolder in subfolders:
    os.makedirs(os.path.join(subfolder,'trim_galore_output'),exist_ok=True)
    trim_galore_dir_list.append(os.path.join(subfolder,'trim_galore_output'))

    os.makedirs(os.path.join(subfolder,'nugen_trim_output'),exist_ok=True)
    nugen_trim_dir_list.append(os.path.join(subfolder,'diversity_trim_output'))

    os.makedirs(os.path.join(subfolder,'bismark_output'),exist_ok=True)
    bismark_dir_list.append(os.path.join(subfolder,'bismark_output'))

    os.makedirs(os.path.join(subfolder,'dedup_output'),exist_ok=True)
    dedup_dir_list.append(os.path.join(subfolder,'dedup_output'))

genome_dir = "/mnt/sda1/jwright/Genomes/20240610_Human/"
bismark_genome_dir = os.path.join(genome_dir,'Bisulfite_Genome')

fastqc_dir = os.path.join(target_dir,"fastqc_output")
os.makedirs(fastqc_dir,exist_ok=True)

methylation_extraction_dir = os.path.join(target_dir,"methylation_extraction_output")
os.makedirs(methylation_extraction_dir,exist_ok=True)

report_dir = os.path.join(target_dir,"bismark_report_output")
os.makedirs(report_dir,exist_ok=True)

summary_dir = os.path.join(target_dir,"bismark_summary_output")
os.makedirs(summary_dir,exist_ok=True)

In [7]:
try:
    genome_fasta_file = (glob.glob(os.path.join(genome_dir,"*.fa")) + glob.glob(os.path.join(genome_dir,"*.fna")))[0]
    print(f"Found genome fasta file at {genome_fasta_file}")
except IndexError:
    print(f"No genome fasta file found in genome directory at {genome_dir}")

try:
    genome_gtf_file = (glob.glob(os.path.join(genome_dir,"*.gtf")) + glob.glob(os.path.join(genome_dir,"*.gff")))[0]
    print(f"Found genome gtf file at {genome_gtf_file}")
except IndexError:
    print(f"No genome gtf file found in genome directory at {genome_dir}")

genome_preparation_cmd = [
    "bismark_genome_preparation",
    "--verbose",
    "--bowtie2",
    "--parallel","9",
    genome_dir
]

if os.path.exists(os.path.join(bismark_genome_dir,'CT_conversion')):
    print(f"Genome already generated in {bismark_genome_dir}")
else:
    try:
        print("Attempting to generate genome index...")
        process = subprocess.run(genome_preparation_cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True)
        print(process.stdout)
        print("Genome index generated!")
    except Exception as e:
        print(e)

Found genome fasta file at /mnt/sda1/jwright/Genomes/20240610_Human/Homo_sapiens.GRCh38.dna.primary_assembly.fa
Found genome gtf file at /mnt/sda1/jwright/Genomes/20240610_Human/Homo_sapiens.GRCh38.112.gtf
Attempting to generate genome index...
Path to genome folder specified as: /mnt/sda1/jwright/Genomes/20240610_Human/
Aligner to be used: >> Bowtie 2 << (user-defined)
Bismark Genome Preparation - Step I: Preparing folders

Created Bisulfite Genome folder /mnt/sda1/jwright/Genomes/20240610_Human/Bisulfite_Genome/
Created Bisulfite Genome folder /mnt/sda1/jwright/Genomes/20240610_Human/Bisulfite_Genome/CT_conversion/
Created Bisulfite Genome folder /mnt/sda1/jwright/Genomes/20240610_Human/Bisulfite_Genome/GA_conversion/
Bismark Genome Preparation - Step II: Bisulfite converting reference genome

conversions performed:
chromosome	C->T	G->A
1	48055043	48111528
10	27639505	27719976
11	27903257	27981801
12	27092804	27182678
13	18839192	18933605
14	18423758	18559033
15	17752941	17825903
16	

In [8]:
for directory in subfolders:
    trim_galore_input_file_read1 = sorted(glob.glob(os.path.join(directory,"*.fastq.gz")))[0]
    trim_galore_input_file_read2 = sorted(glob.glob(os.path.join(directory,"*.fastq.gz")))[2]
    output_dir = os.path.join(directory,'trim_galore_output')

    trim_galore_cmd = [
        "trim_galore",
        "--cores","4",
        "--paired",
        "--gzip",
        "--output_dir",output_dir,
        "-a","AGATCGGAAGAGC",
        "-a2","AAATCAAAAAAAC",
        trim_galore_input_file_read1,
        trim_galore_input_file_read2
    ]

    try:
        trim_galore_file = glob.glob(os.path.join(directory,'trim_galore_output','*_trimming_report.txt'))[0]
        print(f"Found Trim Galore already completed in this directory: {directory}")
    except Exception:
        print(f"Now attempting Trim Galore in: {directory}...")
        process = subprocess.run(trim_galore_cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True)
        print(process.stdout)
    
print("All Trim Galore trimming has completed!")

Now attempting Trim Galore in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/...

Now attempting Trim Galore in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252163/...

Now attempting Trim Galore in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252164/...

Now attempting Trim Galore in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252165/...

Now attempting Trim Galore in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252166/...

Now attempting Trim Galore in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252167/...

All Trim Galore trimming has completed!


In [9]:
for directory in subfolders:
    nugen_input_file_read1 = glob.glob(os.path.join(directory,'trim_galore_output','*1.fq.gz'))[0]
    nugen_input_file_read2 = glob.glob(os.path.join(directory,'trim_galore_output','*2.fq.gz'))[0]
    output_dir = os.path.join(directory,'nugen_trim_output')

    nugen_trim_cmd = [
        "python2",
        "trimRRBSdiversityAdaptCustomers.py",
        "-1",nugen_input_file_read1,
        "-2",nugen_input_file_read2,
        "-o",output_dir
    ]

    nugen_trim_cmd_str = " ".join(map(str,nugen_trim_cmd))

    try:
        nugen_trim_file = glob.glob(os.path.join(directory,'nugen_trim_output','*_trimmed.fq.gz'))[0]
        print(f"Found NuGEN Trimming output in this directory: {directory}")
    except Exception:
        print(f"Now attempting NuGEN trimming in: {directory}...")
        try:
            process = subprocess.run(nugen_trim_cmd_str,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True,shell=True)
            print(process.stdout)
        except subprocess.CalledProcessError as e:
            print("Error:", e)

print("All NuGEN post-trimming has been completed!")

Now attempting NuGEN trimming in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/...
Your files are:
Fwd files:
	/home/jwright/RRBS/RQ025464-Cortese/RS-04252162/trim_galore_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq.gz
Rev files:
	/home/jwright/RRBS/RQ025464-Cortese/RS-04252162/trim_galore_output/RS-04252162_RP-19_RS-04224565_S1_R3_001_val_2.fq.gz
	Done with this pair. there were 48341244 records
	Fwd:  D0:10783308  D1:25240667  D2:7718238  D3:2209616 other=2389415 (total=48341244)

Now attempting NuGEN trimming in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252163/...
Your files are:
Fwd files:
	/home/jwright/RRBS/RQ025464-Cortese/RS-04252163/trim_galore_output/RS-04252163_RP-39_RS-04224566_S2_R1_001_val_1.fq.gz
Rev files:
	/home/jwright/RRBS/RQ025464-Cortese/RS-04252163/trim_galore_output/RS-04252163_RP-39_RS-04224566_S2_R3_001_val_2.fq.gz
	Done with this pair. there were 40783761 records
	Fwd:  D0:8647763  D1:22154223  D2:6347796  D3:1627472 other=2006507 (total=40783761

In [17]:
fastqc_dir

'/home/jwright/RRBS/RQ025464-Cortese/fastqc_output'

In [18]:
fastqc_input_file_list = sorted(glob.glob(os.path.join(target_dir,"*","nugen_trim_output","*_trimmed.fq.gz")))

fastqc_cmd = [
    "fastqc",
    "--threads","20"
    "--outdir",fastqc_dir
]

fastqc_cmd.extend(fastqc_input_file_list)

fastqc_file_list = glob.glob(os.path.join(fastqc_dir,"*.html"))
if len(fastqc_file_list) >= 1:
    print(f"Found FastQC output in this directory: {fastqc_dir}")
else:
    print(f"Now attempting FastQC on these FastQ files:")
    for file in fastqc_input_file_list:
        print(file)
    process = subprocess.run(fastqc_cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True)
    print(process.stdout)

print("FastQC has been completed!")

Now attempting FastQC on these FastQ files:
/home/jwright/RRBS/RQ025464-Cortese/RS-04252162/nugen_trim_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed.fq.gz
/home/jwright/RRBS/RQ025464-Cortese/RS-04252162/nugen_trim_output/RS-04252162_RP-19_RS-04224565_S1_R3_001_val_2.fq_trimmed.fq.gz
/home/jwright/RRBS/RQ025464-Cortese/RS-04252163/nugen_trim_output/RS-04252163_RP-39_RS-04224566_S2_R1_001_val_1.fq_trimmed.fq.gz
/home/jwright/RRBS/RQ025464-Cortese/RS-04252163/nugen_trim_output/RS-04252163_RP-39_RS-04224566_S2_R3_001_val_2.fq_trimmed.fq.gz
/home/jwright/RRBS/RQ025464-Cortese/RS-04252164/nugen_trim_output/RS-04252164_RP-40_RS-04224570_S3_R1_001_val_1.fq_trimmed.fq.gz
/home/jwright/RRBS/RQ025464-Cortese/RS-04252164/nugen_trim_output/RS-04252164_RP-40_RS-04224570_S3_R3_001_val_2.fq_trimmed.fq.gz
/home/jwright/RRBS/RQ025464-Cortese/RS-04252165/nugen_trim_output/RS-04252165_RP-26_RS-04224567_S4_R1_001_val_1.fq_trimmed.fq.gz
/home/jwright/RRBS/RQ025464-Cortese/RS-04252165/nugen

In [2]:
%%bash

multiqc --outdir /home/jwright/RRBS/RQ025464-Cortese/fastqc_output /home/jwright/RRBS/RQ025464-Cortese/fastqc_output


/// ]8;id=182921;https://multiqc.info\MultiQC]8;;\ 🔍 v1.22.2

       file_search | Search path: /home/jwright/RRBS/RQ025464-Cortese/fastqc_output
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 24/24  lm  
            fastqc | Found 12 reports
     write_results | Data        : ../../../mnt/sda1/jwright/RRBS/RQ025464-Cortese/fastqc_output/multiqc_data
     write_results | Report      : ../../../mnt/sda1/jwright/RRBS/RQ025464-Cortese/fastqc_output/multiqc_report.html
           multiqc | MultiQC complete


In [12]:
for directory in subfolders:
    bismark_input_file_read1 = glob.glob(os.path.join(directory,'nugen_trim_output','*1.fq_trimmed.fq.gz'))[0]
    bismark_input_file_read2 = glob.glob(os.path.join(directory,'nugen_trim_output','*2.fq_trimmed.fq.gz'))[0]
    output_dir = os.path.join(directory,'bismark_output')

    bismark_cmd = [
        "bismark",
        "--bowtie2",
        "--gzip",
        "--bam",
        "--p","20",
        "--genome_folder",genome_dir,
        "-1",bismark_input_file_read1,
        "-2",bismark_input_file_read2,
        "--outdir", output_dir
    ]

    try:
        bismark_file = glob.glob(os.path.join(directory,'bismark_output','*fill_in_later.txt'))[0]
        print(f"Found Bismark output in this directory: {directory}")
    except Exception:
        print(f"Now attempting Bismark alignment in: {directory}...")
        try:
            process = subprocess.run(bismark_cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True)
            print(process.stdout)
        except subprocess.CalledProcessError as e:
            print("Error:", e)

Now attempting Bismark alignment in: /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/...
chr 1 (248956422 bp)
chr 10 (133797422 bp)
chr 11 (135086622 bp)
chr 12 (133275309 bp)
chr 13 (114364328 bp)
chr 14 (107043718 bp)
chr 15 (101991189 bp)
chr 16 (90338345 bp)
chr 17 (83257441 bp)
chr 18 (80373285 bp)
chr 19 (58617616 bp)
chr 2 (242193529 bp)
chr 20 (64444167 bp)
chr 21 (46709983 bp)
chr 22 (50818468 bp)
chr 3 (198295559 bp)
chr 4 (190214555 bp)
chr 5 (181538259 bp)
chr 6 (170805979 bp)
chr 7 (159345973 bp)
chr 8 (145138636 bp)
chr 9 (138394717 bp)
chr MT (16569 bp)
chr X (156040895 bp)
chr Y (57227415 bp)
chr KI270728.1 (1872759 bp)
chr KI270727.1 (448248 bp)
chr KI270442.1 (392061 bp)
chr KI270729.1 (280839 bp)
chr GL000225.1 (211173 bp)
chr KI270743.1 (210658 bp)
chr GL000008.2 (209709 bp)
chr GL000009.2 (201709 bp)
chr KI270747.1 (198735 bp)
chr KI270722.1 (194050 bp)
chr GL000194.1 (191469 bp)
chr KI270742.1 (186739 bp)
chr GL000205.2 (185591 bp)
chr GL000195.1 (182896 bp)
chr K

In [46]:
def bam_to_sam(bam_file,output_sam_file):
    process = subprocess.run(["samtools","view","-o",output_sam_file,"-h",bam_file])
    print(process.stdout) if process.stdout else None
    return print(f"Converted {bam_file} to sam file: {output_sam_file}")

def run_strip_bismark_sam(sam_file):
    process = subprocess.run(["./strip_bismark_sam.sh",sam_file])
    print(process.stdout) if process.stdout else None
    return print(f"Processed {sam_file}")

def deduplicate(sam_file,read_file,output_file_name):
    process = subprocess.run(["./nudup.py","-2","-f",read_file,"-o",output_file_name,sam_file])
    print(process.stdout) if process.stdout else None
    return print(f"Removed duplicates from {sam_file}")

# def sam_to_bam(sam_file,output_bam_file):
#     process = subprocess.run(["samtools","view","-o",output_bam_file,"-b",sam_file])
#     print(process.stdout)
#     return print(f"Converted {sam_file} to Bam: {output_bam_file}")

for directory in subfolders:
    rs_number = directory.split('/')[-2]
    bismark_bam_file = glob.glob(os.path.join(directory,'bismark_output','*.bam'))[0]
    temp_sam_file = os.path.join(directory,'dedup_output',f'temporary_{rs_number}.sam')
    test_output_file = os.path.join(directory,'dedup_output',f'deduplicated_{rs_number}.sorted.dedup.bam')

    if os.path.exists(test_output_file):
        print(f'Deduplication already performed in {directory}, skipping to the next sample.\n')

    else:
        print('Starting deduplication for this sample: ' + rs_number)
        print('\nAttempting to start 4-Step Deduplication process')
        print('Converting existing BAM file to SAM file')

        bam_to_sam(bismark_bam_file,temp_sam_file)

        print('\nRunning custom stripping script provided by NuGEN')

        run_strip_bismark_sam(temp_sam_file)

        stripped_sam_file = glob.glob(os.path.join(directory,'dedup_output','*_stripped.sam'))[0]
        read_file_for_deduplication = glob.glob(os.path.join(directory,'*R1_001.fastq.gz'))[0] # Using Read 1 while that works
        output_bam_file = os.path.join(directory,'dedup_output',f'deduplicated_{rs_number}')

        print('\nPerforming NuGEN Deduplication using the following files:')
        print('Stripped SAM file: ' + stripped_sam_file)
        print('Reference Read File: ' + read_file_for_deduplication)
        print('Output BAM file: ' + output_bam_file)

        deduplicate(stripped_sam_file,read_file_for_deduplication,output_bam_file)

        print('\nNow removing any generated SAM files to conserve space. This includes the custom generated stripped file')
        print('Removing these SAM files:')

        sam_file_list = glob.glob(os.path.join(directory,'dedup_output','*.sam'))
        for sam_file in sam_file_list:
            print(sam_file)
            os.remove(sam_file)

        print('SAM Files successfully removed')

        print(f'\nFinished Deduplication for Sample {rs_number}! Now checking to see if further Deduplication is needed.\n')
        
    
print('Deduplication complete!')
    
    

    




Deduplication already performed in /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/, skipping to the next sample.

Deduplication already performed in /home/jwright/RRBS/RQ025464-Cortese/RS-04252163/, skipping to the next sample.

Deduplication already performed in /home/jwright/RRBS/RQ025464-Cortese/RS-04252164/, skipping to the next sample.

Deduplication already performed in /home/jwright/RRBS/RQ025464-Cortese/RS-04252165/, skipping to the next sample.

Deduplication already performed in /home/jwright/RRBS/RQ025464-Cortese/RS-04252166/, skipping to the next sample.

Deduplication already performed in /home/jwright/RRBS/RQ025464-Cortese/RS-04252167/, skipping to the next sample.

Deduplication complete!


In [ ]:
# for directory in subfolders:
#     bismark_input_file_read1 = glob.glob(os.path.join(directory,'nugen_trim_output','*1.fq_trimmed.fq.gz'))[0]
#     bismark_input_file_read2 = glob.glob(os.path.join(directory,'nugen_trim_output','*2.fq_trimmed.fq.gz'))[0]
#     output_dir = os.path.join(directory,'bismark_output')

#     bismark_cmd = [
#         "bismark",
#         "--bowtie2",
#         "--gzip",
#         "--bam",
#         "--p","20",
#         "--genome_folder",genome_dir,
#         "-1",bismark_input_file_read1,
#         "-2",bismark_input_file_read2,
#         "--basename",output_dir
#     ]

#     try:
#         bismark_file = glob.glob(os.path.join(directory,'bismark_output','*fill_in_later.txt'))[0]
#         print(f"Found Bismark output in this directory: {directory}")
#     except Exception:
#         print(f"Now attempting Bismark alignment in: {directory}...")
#         try:
#             process = subprocess.run(bismark_cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True)
#             print(process.stdout)
#         except subprocess.CalledProcessError as e:
#             print("Error:", e)

In [7]:
subfolders

['/home/jwright/RRBS/RQ025464-Cortese/RS-04252162/',
 '/home/jwright/RRBS/RQ025464-Cortese/RS-04252163/',
 '/home/jwright/RRBS/RQ025464-Cortese/RS-04252164/',
 '/home/jwright/RRBS/RQ025464-Cortese/RS-04252165/',
 '/home/jwright/RRBS/RQ025464-Cortese/RS-04252166/',
 '/home/jwright/RRBS/RQ025464-Cortese/RS-04252167/']

In [13]:
bismark_bam_file_list = []
output_dir = os.path.join(target_dir,"methylation_extraction_output")

for subfolder in subfolders:
    bam_file = glob.glob(os.path.join(subfolder,"bismark_output","*.bam"))[0]
    bismark_bam_file_list.append(bam_file)

methylation_extraction_cmd = [
    "bismark_methylation_extractor",
    "--paired-end",
    "--comprehensive",
    "--merge_non_CpG",
    "--output_dir",output_dir,
    "--gzip",
    "--parallel","15",
    "--bedGraph","--remove_spaces",
    "--cytosine_report",
    "--genome_folder",genome_dir
]

methylation_extraction_cmd.extend(bismark_bam_file_list)

if len(os.listdir(output_dir)) > 0:
    print(f"Output files already exist in: {output_dir} \nIf extracting methylation again please remove files and re-run.")
else:
    print("Now performing Bismark Methylation Extraction...")
    process = subprocess.run(methylation_extraction_cmd,stderr=subprocess.PIPE,stdout=subprocess.PIPE,text=True)
    print("Methylation Extraction complete.")

Output files already exist in /home/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output 
If extracting methylation again please remove files and re-run.


In [28]:
for subfolder in subfolders:
    rs_number = os.path.basename(subfolder)

    alignment_report = glob.glob(os.path.join(subfolder,"bismark_output","*.txt"))[0]
    splitting_report = glob.glob(os.path.join(methylation_extraction_dir,f"*{rs_number}**_splitting_report.txt"))[0]
    mbias_report = glob.glob(os.path.join(methylation_extraction_dir,f"*{rs_number}**M-bias.txt"))[0]
    
    report_output_file = os.path.join(target_dir,"bismark_report_output",f"{rs_number}.report.html")

    report_cmd = [
        "bismark2report",
        "--dir",report_dir,
        "--alignment_report", alignment_report,
        "--splitting_report", splitting_report,
        "--mbias_report",mbias_report
    ]

    if os.path.exists(report_output_file):
        print(f"Report already generated: {report_output_file}")
    else:
        print(f"Generating Bismark Report file in this subfolder: {rs_number}")
        print(f"Report command: {" ".join(report_cmd)}")
        process = subprocess.run(report_cmd,stdout=subprocess.PIPE,stderr=subprocess.PIPE,text=True)
        print("Completed report generation, moving to the next subfolder...")
print("Reports for all samples have been generated!")

Generating Bismark Report file in this subfolder: RS-04252162
Report command: bismark2report --dir /home/jwright/RRBS/RQ025464-Cortese/bismark_report_output --alignment_report /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/bismark_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_PE_report.txt --splitting_report /home/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt --mbias_report /home/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe.M-bias.txt
Completed report generation, moving to the next subfolder...
Generating Bismark Report file in this subfolder: RS-04252163
Report command: bismark2report --dir /home/jwright/RRBS/RQ025464-Cortese/bismark_report_output --alignment_report /home/jwright/RRBS/RQ025464-Cortese/RS-04252163/bismark_output/RS-04252163_RP-39_RS-04224566_S2_R1_

In [15]:
%%bash
bismark2report

Found no potential alignment reports in the current directory. Please specify a single Bismark alignment report file using the option '--alignment_report FILE'




  SYNOPSIS:

  This script uses a Bismark alignment report to generate a graphical HTML report page. Optionally, further reports of
  the Bismark suite such as deduplication, methylation extractor splitting or M-bias reports can be specified as well. If several
  Bismark reports are found in the same folder, a separate report will be generated for each of these, whereby the output filename
  will be derived from the Bismark alignment report file. bismark2report attempts to find optional reports automatically based
  on the file basename.


  USAGE: bismark2report [options]


-o/--output <filename>     Name of the output file (optional). If not specified explicitly, the output filename will be derived
                           from the Bismark alignment report file. Specifying an output filename only works if the HTML report is
                           to be generated for a single Bismark alignment report (and potentially additional reports).

--dir                      Output direc

CalledProcessError: Command 'b'bismark2report\n'' returned non-zero exit status 1.

In [4]:
%%bash

bismark_methylation_extractor --paired-end --comprehensive --merge_non_CpG --output_dir /home/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output --gzip --parallel 15 --bedGraph --remove_spaces --cytosine_report --genome_folder /mnt/sda2/jwright/Genomes/20240610_Human \
    /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/bismark_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam \
    /home/jwright/RRBS/RQ025464-Cortese/RS-04252163/bismark_output/RS-04252163_RP-39_RS-04224566_S2_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam \
    /home/jwright/RRBS/RQ025464-Cortese/RS-04252164/bismark_output/RS-04252164_RP-40_RS-04224570_S3_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam \
    /home/jwright/RRBS/RQ025464-Cortese/RS-04252165/bismark_output/RS-04252165_RP-26_RS-04224567_S4_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam \
    /home/jwright/RRBS/RQ025464-Cortese/RS-04252166/bismark_output/RS-04252166_RP-35_RS-04224568_S5_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam \
    /home/jwright/RRBS/RQ025464-Cortese/RS-04252167/bismark_output/RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam


 *** Bismark methylation extractor version v0.24.2 ***

Output will be written into the directory: /mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/
Setting option '--no_overlap' since this is (normally) the right thing to do for paired-end data


Summarising Bismark methylation extractor parameters:
Bismark paired-end SAM format specified (default)
Number of cores to be used: 15
Strand-specific outputs will be skipped. Separate output files for cytosines in CpG, CHG and CHH context will be generated
Merge CHG and CHH context to non-CpG context specified
Output path specified as: /mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/


Summarising bedGraph parameters:
Generating additional output in bedGraph and coverage format
bedGraph format:	<Chromosome> <Start Position> <End Position> <Methylation Percentage>
coverage format:	<Chromosome> <Start Position> <End Position> <Methylation Percentage> <count methylated> <count non-methylated>

Using a

...passed!
Writing result file containing methylation information for C in CpG context to /mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/CpG_context_RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe.txt.gz
Writing result file containing methylation information for C in any other context to /mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/Non_CpG_context_RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe.txt.gz

Now reading in Bismark result file /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/bismark_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam


Now reading in Bismark result file /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/bismark_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam


Now reading in Bismark result file /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/bismark_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq

Now testing Bismark result file >/home/jwright/RRBS/RQ025464-Cortese/RS-04252162/bismark_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bam< for positional sorting (which would be bad...)	All child process successfully finished./mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.1
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.2
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.3
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.4
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-0425216


Processed 30233814 lines in total
Total number of methylation call strings processed: 60467628

Final Cytosine Methylation Report
Total number of C's analysed:	842502117

Total methylated C's in CpG context:	65326555
Total methylated C's in CHG context:	1206153
Total methylated C's in CHH context:	1676356

Total C to T conversions in CpG context:	77635166
Total C to T conversions in CHG context:	224570170
Total C to T conversions in CHH context:	472087717

C methylated in CpG context:	45.7%
C methylated in non-CpG context:	0.4%



Merging individual M-bias reports into overall M-bias statistics from these 15 individual files:


/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.1.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.2.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.3.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.4.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.5.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq_trimmed_bismark_bt2_pe_spl

...


Processed 25897884 lines in total
Total number of methylation call strings processed: 51795768

Final Cytosine Methylation Report
Total number of C's analysed:	724046731

Total methylated C's in CpG context:	57552349
Total methylated C's in CHG context:	1042184
Total methylated C's in CHH context:	1432413

Total C to T conversions in CpG context:	64573500
Total C to T conversions in CHG context:	192851978
Total C to T conversions in CHH context:	406594307

C methylated in CpG context:	47.1%
C methylated in non-CpG context:	0.4%



Merging individual M-bias reports into overall M-bias statistics from these 15 individual files:


/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.1.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.2.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.3.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.4.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe_splitting_report.txt.5.mbias
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe_spl

Determining maximum read lengths for M-Bias plots
Maximum read length of Read 1: 95
Maximum read length of Read 2: 93

Perl module GD::Graph::lines is not installed, skipping drawing M-bias plots (only writing out M-bias plot table)
Determining maximum read lengths for M-Bias plots
Maximum read length of Read 1: 95
Maximum read length of Read 2: 93

Perl module GD::Graph::lines is not installed, skipping drawing M-bias plots (only writing out M-bias plot table)
Deleting unused files ...

/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/CpG_context_RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe.txt.gz contains data ->	kept
/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/Non_CpG_context_RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe.txt.gz contains data ->	kept


Using these input files: /mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/CpG_context_RS-04252167_RP-31_RS-0422456

/mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/CpG_context_RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe.txt.gz



Writing bedGraph to file: RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bedGraph.gz
Also writing out a coverage file including counts methylated and unmethylated residues to file: RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe.bismark.cov.gz

Changed directory to /mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/
Now replacing whitespaces in the sequence ID field of the Bismark methylation extractor output /mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/CpG_context_RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe.txt.gz prior to bedGraph conversion

Attempting to write to file /mnt/sda2/jwright/RRBS/RQ025464-Cortese/methylation_extraction_output/CpG_context_RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bismark_bt2_pe.txt.gz.spaces_removed.txt

Now writing methylation information for file >>CpG_context_RS-04252167_RP-31_RS-04224569_S6_R1_001_val_1.fq_trimmed_bis

In [ ]:
%%bash

./bismark --bowtie2 --gzip --bam --basename test --output_dir /home/jwright/RRBS -p 9 --genome /mnt/sda1/jwright/20240430_human_genome/ /home/jwright/Bismark/test_data.fastq

In [41]:
%%bash

bismark --help


DESCRIPTION

The following is a brief description of command line options and arguments to control the Bismark
bisulfite mapper and methylation caller. Bismark takes in FastA or FastQ files and aligns the
reads to a specified bisulfite genome. Sequence reads are transformed into a bisulfite converted forward strand
version (C->T conversion) or into a bisulfite treated reverse strand (G->A conversion of the forward strand).
Each of these reads are then aligned to bisulfite treated forward strand index of a reference genome
(C->T converted) and a bisulfite treated reverse strand index of the genome (G->A conversion of the
forward strand, by doing this alignments will produce the same positions). These 4 instances of Bowtie 2 or HISAT2
are run in parallel. The sequence file(s) are then read in again sequence by sequence to pull out the original
sequence from the genome and determine if there were any protected C's present or not.

The final output of Bismark is in BAM/SAM format by defau

In [53]:
%%bash

trim_galore --cores 9 --paired --gzip -a AGATCGGAAGAGC -a2 AAATCAAAAAAAC /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/RS-04252162_RP-19_RS-04224565_S1_R1_001.fastq.gz /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/RS-04252162_RP-19_RS-04224565_S1_R3_001.fastq.gz

Using an excessive number of cores has a diminishing return! It is recommended not to exceed 8 cores per trimming process (you asked for 9 cores). Please consider re-specifying
Path to Cutadapt set as: 'cutadapt' (default)
Cutadapt seems to be working fine (tested command 'cutadapt --version')
Cutadapt version: 4.4
Cutadapt seems to be using Python 3! Proceeding with multi-core enabled Cutadapt using 9 cores
Proceeding with 'gzip' for compression. PLEASE NOTE: Using multi-cores for trimming with 'gzip' only has only very limited effect! (see here: https://github.com/FelixKrueger/TrimGalore/issues/16#issuecomment-458557103)
To increase performance, please install 'pigz' and run again

Proceeding with 'gzip' for decompression
To decrease CPU usage of decompression, please install 'igzip' and run again

No quality encoding type selected. Assuming that the data provided uses Sanger encoded Phred scores (default)

Writing report to 'RS-04252162_RP-19_RS-04224565_S1_R1_001.fastq.gz_trimming_

In [18]:
%%bash

bismark_genome_preparation --verbose --bowtie2 --parallel 9 /mnt/sda1/jwright/20240430_human_genome/


Path to genome folder specified as: /mnt/sda1/jwright/20240430_human_genome/


Using 9 threads for the top and bottom strand indexing processes each, so using 18 cores in total


Aligner to be used: >> Bowtie 2 << (user-defined)
Bismark Genome Preparation - Step I: Preparing folders



Writing bisulfite genomes out into a single MFA (multi FastA) file

Bisulfite Genome Indexer version v0.24.2 (last modified: 19 May 2022)



A directory called /mnt/sda1/jwright/20240430_human_genome/Bisulfite_Genome/ already exists. Already existing converted sequences and/or already existing Bowtie 2, HISAT2 or Minimap2) indices will be overwritten!

Bismark Genome Preparation - Step II: Bisulfite converting reference genome



...

Preparing indexing of GA converted genome in /mnt/sda1/jwright/20240430_human_genome/Bisulfite_Genome/GA_conversion/


Child process: Starting to index G->A converted genome with the following command:

bowtie2-build --threads 9   -f genome_mfa.GA_conversion.fa BS_GA

Settings:
  Output files: "BS_GA.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 36
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  genome_mfa.GA_conversion.fa


Building a SMALL index


Reading reference sizes
  Time reading reference sizes: 00:00:17
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time reading reference sizes: 00:00:16
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:12
bmax according to bmaxDivN setting: 86951409
Using parameters --bmax 65213557 --dcv 1024
  Doing ahead-of-time memory usage test
  Time to join reference sequences: 00:00:12
bmax according to bmaxDivN setting: 86951409
Using parameters --bmax 65213557 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 65213557 --dcv 1024
Constructing suffix-array element generator
Building DifferenceCoverSample
  Building sPrime
  Passed!  Constructing with these parameters: --bmax 65213557 --dcv 1024
  Building sPrimeOrder
Constructing suffix-array element generator
Building DifferenceCoverSample
 



Parallel genome indexing complete. Enjoy!



In [20]:
%%bash

./bismark --bowtie2 --gzip --bam --basename test --output_dir /home/jwright/RRBS -p 9 --genome /mnt/sda1/jwright/20240430_human_genome/ /home/jwright/Bismark/test_data.fastq

Bowtie 2 seems to be working fine (tested command 'bowtie2 --version' [2.3.5])
Output format is BAM (default)
Alignments will be written out in BAM format. Samtools found here: '/home/jwright/anaconda3/envs/RRBS/bin/samtools'
Reference genome folder provided is /mnt/sda1/jwright/20240430_human_genome/	(absolute path is '/mnt/sda1/jwright/20240430_human_genome/)'
FastQ format assumed (by default)
Attention: early reports suggested that high values of -p  to have diminishing returns. Please test different values using a small subset of data for your hardware setting.
Each Bowtie 2 instance is going to be run with 9 threads. Please monitor performance closely and tune down if necessary!

Input files to be analysed (in current folder '/home/jwright/Bismark'):
/home/jwright/Bismark/test_data.fastq
Library is assumed to be strand-specific (directional), alignments to strands complementary to the original top or bottom strands will be ignored (i.e. not performed!)
Output will be written into 

chr 1 (248956422 bp)
chr 2 (242193529 bp)
chr 3 (198295559 bp)
chr 4 (190214555 bp)
chr 5 (181538259 bp)
chr 6 (170805979 bp)
chr 7 (159345973 bp)
chr 8 (145138636 bp)
chr 9 (138394717 bp)
chr 10 (133797422 bp)
chr 11 (135086622 bp)
chr 12 (133275309 bp)
chr 13 (114364328 bp)
chr 14 (107043718 bp)
chr 15 (101991189 bp)
chr 16 (90338345 bp)
chr 17 (83257441 bp)
chr 18 (80373285 bp)
chr 19 (58617616 bp)
chr 20 (64444167 bp)
chr 21 (46709983 bp)
chr 22 (50818468 bp)
chr X (156040895 bp)
chr Y (57227415 bp)
chr MT (16569 bp)
chr HG76_PATCH (6367528 bp)
chr HG2365_PATCH (5500449 bp)
chr HSCHR15_4_CTG8 (5161414 bp)
chr HSCHR6_MHC_SSTO_CTG1 (4929269 bp)
chr HSCHR6_MHC_MCF_CTG1 (4827813 bp)
chr HSCHR6_MHC_COX_CTG1 (4795265 bp)
chr HSCHR6_MHC_MANN_CTG1 (4677643 bp)
chr HSCHR6_MHC_APD_CTG1 (4672374 bp)
chr HSCHR6_MHC_QBL_CTG1 (4606388 bp)
chr HSCHR6_MHC_DBB_CTG1 (4604811 bp)
chr HSCHR17_7_CTG4 (2877074 bp)
chr HSCHR16_1_CTG1 (2659700 bp)
chr HSCHR15_6_CTG8 (2365364 bp)
chr HG926_PATCH (1927115 b

Single-core mode: setting pid to 1

Single-end alignments will be performed

Input file is in FastQ format
Writing a C -> T converted version of the input file test_data.fastq to test_data.fastq_C_to_T.fastq.gz

Created C -> T converted version of the FastQ file test_data.fastq (10000 sequences in total)

Input file is test_data.fastq_C_to_T.fastq.gz (FastQ)

Now running 2 instances of Bowtie 2 against the bisulfite genome of /mnt/sda1/jwright/20240430_human_genome/ with the specified options: -q --score-min L,0,-0.2 -p 9 --reorder --ignore-quals

Now starting the Bowtie 2 aligner for CTreadCTgenome (reading in sequences from test_data.fastq_C_to_T.fastq.gz with options -q --score-min L,0,-0.2 -p 9 --reorder --ignore-quals --norc)
Using Bowtie 2 index: /mnt/sda1/jwright/20240430_human_genome/Bisulfite_Genome/CT_conversion/BS_CT

Found first alignment:	SRR020138.15024316_SALK_2029:7:100:1672:1790_length=86	4	*	0	0	*	*	0	0	TTTTTAGAAAGTTTATATGGAAATTAAGTTTTTTGTATATGTTTGTAAAG	BCB?).4'324&1;

Processed 10000 sequences in total




Successfully deleted the temporary file test_data.fastq_C_to_T.fastq.gz

Final Alignment report
Sequences analysed in total:	10000
Number of alignments with a unique best hit from the different alignments:	4718
Mapping efficiency:	47.2%

Final Cytosine Methylation Report
Total number of C's analysed:	38652

Total methylated C's in CpG context:	1305
Total methylated C's in CHG context:	20
Total methylated C's in CHH context:	95
Total methylated C's in Unknown context:	0

Total unmethylated C's in CpG context:	652
Total unmethylated C's in CHG context:	9634
Total unmethylated C's in CHH context:	26946
Total unmethylated C's in Unknown context:	0

C methylated in CpG context:	66.7%
C methylated in CHG context:	0.2%
C methylated in CHH context:	0.4%
Can't determine percentage of methylated Cs in Unknown context (CN or CHN) if value was 0




Sequences with no alignments under any condition:	4172
Sequences did not map uniquely:	1110
Sequences which were discarded because genomic sequence could not be extracted:	0

Number of sequences with unique best (first) alignment came from the bowtie output:
CT/CT:	2368	((converted) top strand)
CT/GA:	2350	((converted) bottom strand)
GA/CT:	0	(complementary to (converted) top strand)
GA/GA:	0	(complementary to (converted) bottom strand)

Number of alignments to (merely theoretical) complementary strands being rejected in total:	0



Bismark completed in 0d 0h 0m 27s

Bismark run complete



In [40]:
%%bash

bismark --help


DESCRIPTION

The following is a brief description of command line options and arguments to control the Bismark
bisulfite mapper and methylation caller. Bismark takes in FastA or FastQ files and aligns the
reads to a specified bisulfite genome. Sequence reads are transformed into a bisulfite converted forward strand
version (C->T conversion) or into a bisulfite treated reverse strand (G->A conversion of the forward strand).
Each of these reads are then aligned to bisulfite treated forward strand index of a reference genome
(C->T converted) and a bisulfite treated reverse strand index of the genome (G->A conversion of the
forward strand, by doing this alignments will produce the same positions). These 4 instances of Bowtie 2 or HISAT2
are run in parallel. The sequence file(s) are then read in again sequence by sequence to pull out the original
sequence from the genome and determine if there were any protected C's present or not.

The final output of Bismark is in BAM/SAM format by defau

In [22]:
%%bash

python2 trimRRBSdiversityAdaptCustomers.py -1 /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/trim_galore_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq.gz -2 /home/jwright/RRBS/RQ025464-Cortese/RS-04252162/trim_galore_output/RS-04252162_RP-19_RS-04224565_S1_R3_001_val_2.fq.gz

Your files are:
Fwd files:
	/home/jwright/RRBS/RQ025464-Cortese/RS-04252162/trim_galore_output/RS-04252162_RP-19_RS-04224565_S1_R1_001_val_1.fq.gz
Rev files:
	/home/jwright/RRBS/RQ025464-Cortese/RS-04252162/trim_galore_output/RS-04252162_RP-19_RS-04224565_S1_R3_001_val_2.fq.gz
